In [95]:
import nltk 

class NamedEntityTagger(nltk.TaggerI):
    def __init__(self,train_sents):
        train_set=[]
        for sentence in train_sents:
            untagged_sent = [(word, tag) for (word, tag, ne_tag) in sentence]
            history = []
            for i, (word, tag, ne_tag) in enumerate(sentence):
                featureset = ne_features(untagged_sent, i, history)
                train_set.append( (featureset, ne_tag) ) 
                history.append(ne_tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
        
    def tag(self, sentence):
        history = []
        for i, (word, tag) in enumerate(sentence):
            featureset = ne_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [99]:
def ne_features(sentence, i, history):
    word, pos = sentence[i]
    return {"word": word}

In [100]:
from nltk.corpus import conll2002

## Ejemplo de uso que deseamos

# Instanciar el tagger 
# - Usa el conjunto de entrenamiento etiquetado
# - Extraer características
# - Entrenar el modelo
train_sentences = conll2002.iob_sents('esp.train')
nerctagger = NamedEntityTagger(train_sentences)

test_sentences = conll2002.iob_sents('esp.testa')
test_sentence = [(word,tag) for (word,tag, ne_tag) in test_sentences[107]]

# Etiquetar oraciones con la tercera columna (ne_tag) dadas las otras dos (word, tag)
# - Extraer características 
# - Usar el modelo entrenado para inferir la nueva etiqueta
# - Otros pasos?
nerctagger.tag(test_sentence)

[((u'El', u'DA'), u'O'),
 ((u'miembro', u'NC'), u'O'),
 ((u'de', u'SP'), u'O'),
 ((u'la', u'DA'), u'O'),
 ((u'Comisi\xf3n', u'NC'), u'B-ORG'),
 ((u'Regional', u'AQ'), u'I-ORG'),
 ((u'de', u'SP'), u'O'),
 ((u'UCE', u'VMI'), u'B-ORG'),
 ((u'Emilio', u'NC'), u'B-PER'),
 ((u'Guerrero', u'NC'), u'I-PER'),
 ((u'expondr\xe1', u'VMI'), u'O'),
 ((u'la', u'DA'), u'O'),
 ((u'posici\xf3n', u'NC'), u'O'),
 ((u'de', u'SP'), u'O'),
 ((u'esta', u'DD'), u'O'),
 ((u'organizaci\xf3n', u'NC'), u'O'),
 ((u'agraria', u'AQ'), u'O'),
 ((u'respecto', u'NC'), u'O'),
 ((u'a', u'SP'), u'O'),
 ((u'la', u'DA'), u'O'),
 ((u'pr\xf3xima', u'AQ'), u'O'),
 ((u'campa\xf1a', u'NC'), u'O'),
 ((u'de', u'SP'), u'O'),
 ((u'tomate', u'NC'), u'O'),
 ((u'.', u'Fp'), u'O')]

In [16]:
tagged_sentence = nerctagger.tag(test_sentence)
[(ne_tag) for (pair,ne_tag) in tagged_sentence]

[u'O',
 u'O',
 u'O',
 u'O',
 u'B-ORG',
 u'I-ORG',
 u'O',
 u'B-ORG',
 u'B-PER',
 u'I-PER',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O']

In [17]:
[(ne_tag) for (word,tag, ne_tag) in test_sentences[107]]

[u'O',
 u'O',
 u'O',
 u'O',
 u'B-ORG',
 u'I-ORG',
 u'I-ORG',
 u'I-ORG',
 u'B-PER',
 u'I-PER',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O',
 u'O']

In [26]:
zip([(ne_tag) for (word,tag, ne_tag) in test_sentences[107]], [(ne_tag) for (pair,ne_tag) in tagged_sentence] )

[(u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'B-ORG', u'B-ORG'),
 (u'I-ORG', u'I-ORG'),
 (u'I-ORG', u'O'),
 (u'I-ORG', u'B-ORG'),
 (u'B-PER', u'B-PER'),
 (u'I-PER', u'I-PER'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O'),
 (u'O', u'O')]

In [27]:
import collections 

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (word,tag,label) in enumerate(test_sentences[107]):
        refsets[label].add(i)

for i, (pair,predicted) in enumerate(tagged_sentence):
        testsets[predicted].add(i)

label_type='I-ORG'
print 'precision:', nltk.metrics.precision(refsets[label_type], testsets[label_type])
print 'recall:', nltk.metrics.recall(refsets[label_type], testsets[label_type])
print 'F-measure:', nltk.metrics.f_measure(refsets[label_type], testsets[label_type])        

pos precision: 1.0
pos recall: 0.333333333333
pos F-measure: 0.5


In [33]:
for label_type in ['B-LOC','I-LOC','B-ORG','I-ORG','B-PER','I-PER']:
    print 'precision(%s):' % label_type, nltk.metrics.precision(refsets[label_type], testsets[label_type])
    print 'recall(%s):' % label_type, nltk.metrics.recall(refsets[label_type], testsets[label_type])
    print 'F-measure(%s):' % label_type, nltk.metrics.f_measure(refsets[label_type], testsets[label_type])
    print '\n'

precision(B-LOC): None
recall(B-LOC): None
F-measure(B-LOC): None


precision(I-LOC): None
recall(I-LOC): None
F-measure(I-LOC): None


precision(B-ORG): 0.5
recall(B-ORG): 1.0
F-measure(B-ORG): 0.666666666667


precision(I-ORG): 1.0
recall(I-ORG): 0.333333333333
F-measure(I-ORG): 0.5


precision(B-PER): 1.0
recall(B-PER): 1.0
F-measure(B-PER): 1.0


precision(I-PER): 1.0
recall(I-PER): 1.0
F-measure(I-PER): 1.0




In [101]:

def eval(nerctagger, test_sentences):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    i = 0
    for test_sentence in test_sentences:
        tagged_sentence = nerctagger.tag([(word,tag) for (word,tag, ne_tag) in test_sentence])
        for ((word,tag,label),(pair,predicted)) in zip(test_sentence,tagged_sentence):
            refsets[label].add(i)
            testsets[predicted].add(i)
            i = i+1

    tags = ['B-LOC','I-LOC','B-ORG','I-ORG','B-PER','I-PER']
    
    (ma_precision, ma_recall, ma_fmeasure) = (0,0,0)
    for label_type in tags:
        precision = nltk.metrics.precision(refsets[label_type], testsets[label_type])
        recall = nltk.metrics.recall(refsets[label_type], testsets[label_type])
        fmeasure = nltk.metrics.f_measure(refsets[label_type], testsets[label_type])
        print 'precision(%s):' % label_type, precision 
        print 'recall(%s):' % label_type, recall 
        print 'F-measure(%s):' % label_type, fmeasure
        print '\n'
        ma_precision += precision
        ma_recall += recall
        ma_fmeasure += fmeasure
        
        
    print "--------------------------------------------------------------------------------"
    print "Precision (Ma):", ma_precision/len(tags)  
    print "Recall (Ma):", ma_recall/len(tags)
    print "F-measure (Ma):", ma_fmeasure/len(tags)
    print "--------------------------------------------------------------------------------"

In [98]:
eval(nerctagger, test_sentences)

precision(B-LOC): 0.636925795053
recall(B-LOC): 0.732723577236
F-measure(B-LOC): 0.681474480151


precision(I-LOC): 0.747967479675
recall(I-LOC): 0.272997032641
F-measure(I-LOC): 0.4


precision(B-ORG): 0.806774441878
recall(B-ORG): 0.616470588235
F-measure(B-ORG): 0.698899633211


precision(I-ORG): 0.617224880383
recall(I-ORG): 0.188872620791
F-measure(I-ORG): 0.289237668161


precision(B-PER): 0.738486842105
recall(B-PER): 0.367430441899
F-measure(B-PER): 0.490710382514


precision(I-PER): 0.582191780822
recall(I-PER): 0.197904540163
F-measure(I-PER): 0.295395308427


--------------------------------------------------------------------------------
Precision (Ma): 0.688261869986
Recall (Ma): 0.396066466827
F-measure (Ma): 0.475952912077
--------------------------------------------------------------------------------
